<a href="https://colab.research.google.com/github/neohack22/ML-engineering/blob/master/02b-agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents for Amazon Bedrock - create agent

This notebook provides sample code for building an Agent for Amazon Bedrock that has an Action Group attached to it.

### Use Case
We will create a assistant that can help you as any chatbot, but with a plus, you will be able to get weather for a City !

### Notebook Walk-through

In this notebook we will:
- Create a lambda function that get weather
- Create an agent
- Create an action group and associate it with the agent
- Test the agent invocation


### Pre-requisites
This notebook requires permissions to:
- create and delete Amazon IAM roles
- create lambda functions
- access Amazon Bedrock

If running on SageMaker Studio, you should add the following managed policies to your role:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess


## Setup
Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries loaded

Let's now import the necessary libraries and initiate the required boto3 clients

In [ ]:
import time
import boto3
import logging
import uuid

from utils.agent import create_agent_role, create_lambda_role
from utils.agent import create_lambda, invoke_agent_helper

### Explication des importations

1. **Bibliothèques standard** :
   - `import time` : Cette bibliothèque fournit des fonctions pour travailler avec le temps. Elle peut être utilisée pour faire des pauses dans l'exécution d'un programme ou pour obtenir l'heure actuelle.
   - `import logging` : Cette bibliothèque permet de générer des messages de log pour le débogage et le suivi des événements dans votre application.
   - `import uuid` : Cette bibliothèque est utilisée pour générer des identifiants uniques. Elle est souvent utilisée pour créer des clés ou des identifiants pour les objets dans une application.

2. **Bibliothèque Boto3** :
   - `import boto3` : Boto3 est le SDK (Software Development Kit) d'Amazon Web Services (AWS) pour Python. Il permet d'interagir avec les services AWS, tels que S3, EC2, Lambda, etc.

3. **Modules personnalisés** :
   - `from utils.agent import create_agent_role, create_lambda_role` : Ici, deux fonctions `create_agent_role` et `create_lambda_role` sont importées depuis un module local `utils.agent`. Ces fonctions sont probablement utilisées pour créer des rôles AWS spécifiques pour des agents ou des fonctions Lambda.
   - `from utils.agent import create_lambda, invoke_agent_helper` : De même, ces deux fonctions `create_lambda` et `invoke_agent_helper` sont également importées. `create_lambda` peut être utilisée pour déployer une fonction Lambda, tandis que `invoke_agent_helper` pourrait être utilisée pour invoquer un agent qui aide à exécuter certaines tâches.

### Exemple de code

Voici le code structuré en utilisant la syntaxe markdown :

```python
import time
import boto3
import logging
import uuid

from utils.agent import create_agent_role, create_lambda_role
from utils.agent import create_lambda, invoke_agent_helper
```

### Conclusion

Ce code établit les bases nécessaires pour interagir avec AWS et gérer des rôles ainsi que des fonctions Lambda, tout en utilisant des outils de journalisation et de gestion du temps.

In [ ]:
session = boto3.session.Session()

In [ ]:
#Clients
s3_client = session.client('s3',)
sts_client = session.client('sts',)
region = session.region_name


In [ ]:
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = session.client('bedrock-agent',)
bedrock_agent_runtime_client = session.client('bedrock-agent-runtime',)
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

Le code que vous avez partagé inclut des appels à des clients AWS et à la configuration du journalage. Voici une explication détaillée de chaque ligne et de son usage.

### Explication du code

1. **Récupération de l'identité de l'appelant** :
   ```python
   account_id = sts_client.get_caller_identity()["Account"]
   ```
   - Cette ligne utilise le client STS (Security Token Service) pour obtenir l'identité de l'appelant. La méthode `get_caller_identity` retourne un dictionnaire contenant des informations sur l'identité de l'appelant, y compris l'ID du compte AWS. On extrait ensuite l'ID du compte en utilisant `["Account"]`.

2. **Création de clients pour le service Bedrock** :
   ```python
   bedrock_agent_client = session.client('bedrock-agent')
   bedrock_agent_runtime_client = session.client('bedrock-agent-runtime')
   ```
   - Ces lignes créent deux clients pour interagir avec les services Bedrock d'AWS. `bedrock-agent` et `bedrock-agent-runtime` sont des services spécifiques fournis par AWS, et les clients permettent de faire des appels à ces services.

3. **Configuration du journalage** :
   ```python
   logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
   logger = logging.getLogger(__name__)
   ```
   - Ici, la configuration de base du journalage est mise en place. Le format spécifie comment les messages de log seront affichés, incluant la date et l'heure, le numéro de processus, le nom du fichier et le niveau de log.
   - `logger = logging.getLogger(__name__)` crée un logger spécifique pour le module actuel, permettant de générer des messages de log qui peuvent être filtrés ou redirigés en fonction de leur provenance.

4. **Variables région et ID de compte** :
   ```python
   region, account_id
   ```
   - Cette ligne semble incomplète car elle ne fait rien de concret avec les variables `region` et `account_id`. Il serait probablement nécessaire d'utiliser ces variables pour des opérations ultérieures, par exemple, en les affichant ou en les utilisant dans des appels à des services AWS.

### Exemple de code

Voici le code structuré en utilisant la syntaxe markdown :

```python
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = session.client('bedrock-agent')
bedrock_agent_runtime_client = session.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id
```

### Conclusion

Ce code établit une connexion aux services AWS, configure le journalage pour le débogage et la surveillance, et récupère des informations sur le compte actuel. Si vous avez des questions supplémentaires ou si vous souhaitez explorer un aspect particulier de ce code, n'hésitez pas à demander !

### Setting up Agent's information

We will now set the variables that define our agent:

- **agent_name**: provides the name of the agent to be created
- **agent_description**: the description of the agent used to display the agents list on the console. This description is **not** part of the agent's prompts
- **agent_instruction**: the instructions of what the agent **should** and **should not** do. This description is part of the agent's prompt and is used during the agent's invocation
- **agent_action_group_name**: the action group name used on the definition of the agent's action.
- **agent_action_group_description:**: the description of the action group name used on the UI to list the action groups. This description is **not** used by the agent's prompts

In [ ]:
suffix = f"{region}-{account_id}"
agent_name = "weather-agent"
agent_bedrock_allow_policy_name = f"{agent_name}"
agent_role_name = f"AmazonBedrockExecutionRoleForAgents_{agent_name}"

agent_description = "This agent provides weather information for a given city"
agent_instruction = """
The agent should be capable of providing accurate weather information, including current conditions and forecasts, when prompted.
The agent must interpret weather data to give practical advice and answer weather-related queries.
However, it's crucial that the agent maintains its ability to engage in a wide range of topics beyond weather.
It should seamlessly switch between weather information and other subjects, using context to determine the nature of each query.
Above all, the agent should remember that while it's capable of providing weather information, this is just one of its many functions, and it should always be prepared to engage with any topic or task presented by the user.
"""

agent_action_group_description = """
This agent provides weather information for a given city.
"""

agent_action_group_name = "weather-action-group"

### Select Foundation Model
You can find more information about the supported foundation models [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-supported.html)

In [ ]:
agent_foundation_model = "anthropic.claude-3-haiku-20240307-v1:0"

### Creating Lambda Function

Next we will create the [AWS Lambda](https://aws.amazon.com/lambda/) function that executes the actions for our agent. This lambda function will have 1 action:
* ```get_weather(city)```: returns the weather for the city


The `lambda_handler` receives the `event` from the agent and the `event` contains information about the `function` to be executed and its `parameters`.

A `functionResponse` is returned by the lambda function with the response body having a `TEXT` field.

You can find more information on how to set your agent lambda function [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html).

Let's first write the code of the lambda function to the `lambda_function.py` file

In [ ]:
lambda_function_name = f'{agent_name}-lambda'

In [ ]:
%%writefile annex/agent/lambda_function.py
import json
import uuid
import boto3
import urllib.request
from urllib.parse import quote


def lambda_handler(event, _):
    """
    This function gets the weather information for a given city
    """

    # Extract info from the event
    actionGroup = event.get('actionGroup', '')
    function = event.get('function', '')
    city = event['parameters'][0]['value']
    encoded_city = quote(city)

    # Get the location data based on the city
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json'
    with urllib.request.urlopen(url) as response:
        location_data = json.loads(response.read().decode())
        if not location_data['results']:
            return {"error": "City not found"}

        lat = location_data['results'][0]['latitude']
        lon = location_data['results'][0]['longitude']

    # Get the weather data based on the location
    weather_url = f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m&daily=weather_code,temperature_2m_max,temperature_2m_min&timezone=auto'
    with urllib.request.urlopen(weather_url) as response:
        weather_data = json.loads(response.read().decode())

    current = weather_data['current']
    daily = weather_data['daily']

    # Prepare the response
    weather_codes = {
        0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
        45: "Fog", 48: "Depositing rime fog",
        51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
        61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
        71: "Slight snow fall", 73: "Moderate snow fall", 75: "Heavy snow fall",
        77: "Snow grains", 80: "Slight rain showers", 81: "Moderate rain showers",
        82: "Violent rain showers", 85: "Slight snow showers", 86: "Heavy snow showers",
        95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
    }
    response_core =  {
        'temperature': current['temperature_2m'],
        'condition': weather_codes.get(current['weather_code'], "Unknown"),
        'humidity': current['relative_humidity_2m'],
        'wind_speed': current['wind_speed_10m'],
        'forecast_max': daily['temperature_2m_max'][0],
        'forecast_min': daily['temperature_2m_min'][0],
        'forecast_condition': weather_codes.get(daily['weather_code'][0], "Unknown")
    }

    responseBody = {'TEXT': {'body': json.dumps(response_core)}}
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }
    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}

    return function_response



Le code que vous avez partagé est une fonction Lambda AWS qui récupère des informations météorologiques pour une ville donnée. Voici une explication détaillée de chaque partie du code.

### Explication du code

1. **Importations** :
   ```python
   import json
   import uuid
   import boto3
   import urllib.request
   from urllib.parse import quote
   ```
   - Ces lignes importent les bibliothèques nécessaires pour manipuler les données JSON, générer des UUID, interagir avec AWS, et faire des requêtes HTTP.

2. **Définition de la fonction `lambda_handler`** :
   ```python
   def lambda_handler(event, _):
   ```
   - C'est le point d'entrée de la fonction Lambda. Elle prend en entrée un événement (généralement un dictionnaire) et un contexte (non utilisé ici).

3. **Extraction des informations de l'événement** :
   ```python
   actionGroup = event.get('actionGroup', '')
   function = event.get('function', '')
   city = event['parameters'][0]['value']
   encoded_city = quote(city)
   ```
   - Cette section extrait le groupe d'actions, la fonction, et le nom de la ville à partir de l'événement. La ville est ensuite encodée pour être utilisée dans une URL.

4. **Obtenir les données de localisation** :
   ```python
   url = f'https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json'
   with urllib.request.urlopen(url) as response:
       location_data = json.loads(response.read().decode())
       if not location_data['results']:
           return {"error": "City not found"}
       
       lat = location_data['results'][0]['latitude']
       lon = location_data['results'][0]['longitude']
   ```
   - Une requête est faite à l'API de géocodage pour obtenir les coordonnées (latitude et longitude) de la ville spécifiée. Si la ville n'est pas trouvée, un message d'erreur est retourné.

5. **Obtenir les données météorologiques** :
   ```python
   weather_url = f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m&daily=weather_code,temperature_2m_max,temperature_2m_min&timezone=auto'
   with urllib.request.urlopen(weather_url) as response:
       weather_data = json.loads(response.read().decode())
   ```
   - Une seconde requête est faite à l'API météorologique pour récupérer les informations météo en utilisant les coordonnées obtenues.

6. **Préparation de la réponse** :
   ```python
   weather_codes = {
       0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
       45: "Fog", 48: "Depositing rime fog",
       51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
       61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
       71: "Slight snow fall", 73: "Moderate snow fall", 75: "Heavy snow fall",
       77: "Snow grains", 80: "Slight rain showers", 81: "Moderate rain showers",
       82: "Violent rain showers", 85: "Slight snow showers", 86: "Heavy snow showers",
       95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
   }
   response_core =  {   
       'temperature': current['temperature_2m'],
       'condition': weather_codes.get(current['weather_code'], "Unknown"),
       'humidity': current['relative_humidity_2m'],
       'wind_speed': current['wind_speed_10m'],
       'forecast_max': daily['temperature_2m_max'][0],
       'forecast_min': daily['temperature_2m_min'][0],
       'forecast_condition': weather_codes.get(daily['weather_code'][0], "Unknown")
   }
   ```
   - Les données météo sont préparées dans un dictionnaire `response_core`, qui inclut la température, l'humidité, la vitesse du vent, ainsi que les conditions météorologiques.

7. **Construction de la réponse finale** :
   ```python
   responseBody = {'TEXT': {'body': json.dumps(response_core)}}
   action_response = {
       'actionGroup': actionGroup,
       'function': function,
       'functionResponse': {
           'responseBody': responseBody
       }
   }
   function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
   ```
   - La réponse finale est formatée selon la structure attendue, incluant le corps de la réponse et d'autres métadonnées.

8. **Retour de la réponse** :
   ```python
   return function_response
   ```
   - La fonction retourne la réponse formatée.

### Exemple de code

Voici le code structuré avec la syntaxe markdown :

```python
%%writefile annex/agent/lambda_function.py
import json
import uuid
import boto3
import urllib.request
from urllib.parse import quote

def lambda_handler(event, _):
    """
    This function gets the weather information for a given city
    """

    # Extract info from the event
    actionGroup = event.get('actionGroup', '')
    function = event.get('function', '')
    city = event['parameters'][0]['value']
    encoded_city = quote(city)

    # Get the location data based on the city
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json'
    with urllib.request.urlopen(url) as response:
        location_data = json.loads(response.read().decode())
        if not location_data['results']:
            return {"error": "City not found"}
        
        lat = location_data['results'][0]['latitude']
        lon = location_data['results'][0]['longitude']

    # Get the weather data based on the location
    weather_url = f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m&daily=weather_code,temperature_2m_max,temperature_2m_min&timezone=auto'
    with urllib.request.urlopen(weather_url) as response:
        weather_data = json.loads(response.read().decode())

    current = weather_data['current']
    daily = weather_data['daily']

    # Prepare the response
    weather_codes = {
        0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
        45: "Fog", 48: "Depositing rime fog",
        51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
        61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
        71: "Slight snow fall", 73: "Moderate snow fall", 75: "Heavy snow fall",
        77: "Snow grains", 80: "Slight rain showers", 81: "Moderate rain showers",
        82: "Violent rain showers", 85: "Slight snow showers", 86: "Heavy snow showers",
        95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
    }
    response_core =  {   
        'temperature': current['temperature_2m'],
        'condition': weather_codes.get(current['weather_code'], "Unknown"),
        'humidity': current['relative_humidity_2m'],
        'wind_speed': current['wind_speed_10m'],
        'forecast_max': daily['temperature_2m_max'][0],
        'forecast_min': daily['temperature_2m_min'][0],
        'forecast_condition': weather_codes.get(daily['weather_code'][0], "Unknown")
    }

    responseBody = {'TEXT': {'body': json.dumps(response_core)}}
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }
    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}

    return function_response
```

### Conclusion

Cette fonction Lambda est bien structurée pour interagir avec des API externes et fournir des informations météorologiques. Si vous avez des questions supplémentaires ou besoin d'aide sur un aspect particulier du code, n'hésitez pas à demander !

We Create the log group to store the function logs

In [ ]:
logs_client = session.client("logs")
log_group_name = f"/aws/lambda/{lambda_function_name}"

# Create the log group
# Check if the log group already exists
response = logs_client.describe_log_groups(logGroupNamePrefix=log_group_name)
if any(group['logGroupName'] == log_group_name for group in response['logGroups']):
    logger.info(f"Log group '{log_group_name}' already exists.")
else:
    # If log group does not exist, create it
    logs_client.create_log_group(logGroupName=log_group_name)
    logger.info(f"Log group '{log_group_name}' created successfully.")

Le code que vous avez fourni est destiné à interagir avec Amazon CloudWatch Logs pour créer un groupe de journaux (log group) pour une fonction Lambda AWS. Voici une explication détaillée du code :

### Explication du code

1. **Création du client Logs** :
   ```python
   logs_client = session.client("logs")
   log_group_name = f"/aws/lambda/{lambda_function_name}"
   ```
   - Ici, un client CloudWatch Logs est créé à partir d'une session AWS existante. Le nom du groupe de journaux est construit dynamiquement en utilisant le nom de la fonction Lambda.

2. **Vérification de l'existence du groupe de journaux** :
   ```python
   response = logs_client.describe_log_groups(logGroupNamePrefix=log_group_name)
   if any(group['logGroupName'] == log_group_name for group in response['logGroups']):
       logger.info(f"Log group '{log_group_name}' already exists.")
   ```
   - Cette partie du code appelle `describe_log_groups` pour récupérer les groupes de journaux qui commencent par le préfixe `log_group_name`. Il vérifie ensuite si le groupe de journaux existe déjà dans la réponse. Si oui, un message est enregistré pour indiquer que le groupe existe déjà.

3. **Création du groupe de journaux** :
   ```python
   else:
       logs_client.create_log_group(logGroupName=log_group_name)
       logger.info(f"Log group '{log_group_name}' created successfully.")
   ```
   - Si le groupe de journaux n'existe pas, il est créé avec la méthode `create_log_group`. Un message est ensuite enregistré pour indiquer que le groupe de journaux a été créé avec succès.

### Exemple de code

Voici le code structuré avec la syntaxe markdown :

```python
logs_client = session.client("logs")
log_group_name = f"/aws/lambda/{lambda_function_name}"

# Create the log group
# Check if the log group already exists
response = logs_client.describe_log_groups(logGroupNamePrefix=log_group_name)
if any(group['logGroupName'] == log_group_name for group in response['logGroups']):
    logger.info(f"Log group '{log_group_name}' already exists.")
else:
    # If log group does not exist, create it
    logs_client.create_log_group(logGroupName=log_group_name)
    logger.info(f"Log group '{log_group_name}' created successfully.")
```

### Conclusion

Ce code est un bon exemple de la façon dont on peut interagir avec les services AWS en utilisant le SDK `boto3`. Il vérifie d'abord si un groupe de journaux existe avant de tenter de le créer, ce qui évite des erreurs potentielles. Si vous avez des questions supplémentaires ou si vous souhaitez plus de détails sur un aspect spécifique, n'hésitez pas à demander !Le code que vous avez partagé est un extrait typique d'un script Python utilisé pour interagir avec AWS CloudWatch Logs afin de gérer les groupes de journaux (log groups) associés à une fonction Lambda. Voici une analyse détaillée du fonctionnement de ce code, ainsi que des explications supplémentaires sur son contexte et son utilisation.

### Explication du Code

1. **Création du Client de Logs** :
   ```python
   logs_client = session.client("logs")
   ```
   - Ici, un client pour le service AWS CloudWatch Logs est créé en utilisant une session Boto3. Boto3 est la bibliothèque Python pour interagir avec les services AWS. Cela permet à votre application de faire des appels à l'API CloudWatch Logs pour gérer les groupes de journaux, les flux de journaux (log streams), et d'autres ressources.

2. **Nom du Groupe de Journaux** :
   ```python
   log_group_name = f"/aws/lambda/{lambda_function_name}"
   ```
   - Cette ligne définit le nom du groupe de journaux basé sur le nom de votre fonction Lambda. Les groupes de journaux pour AWS Lambda suivent généralement ce format, où `/aws/lambda/` est le préfixe standard, suivi du nom de la fonction. Cela permet de garder les logs organisés et facilement accessibles.

3. **Vérification de l'Existence du Groupe de Journaux** :
   ```python
   response = logs_client.describe_log_groups(logGroupNamePrefix=log_group_name)
   if any(group['logGroupName'] == log_group_name for group in response['logGroups']):
       logger.info(f"Log group '{log_group_name}' already exists.")
   ```
   - Ici, le code appelle `describe_log_groups` pour lister les groupes de journaux existants et vérifier si le groupe de journaux que vous essayez de créer existe déjà. Cette approche évite d'essayer de créer un groupe de journaux qui existe déjà, ce qui entraînerait une erreur.

4. **Création du Groupe de Journaux** :
   ```python
   else:
       logs_client.create_log_group(logGroupName=log_group_name)
       logger.info(f"Log group '{log_group_name}' created successfully.")
   ```
   - Si le groupe de journaux n'existe pas, le code appelle `create_log_group` pour le créer. Ensuite, un message est consigné pour indiquer que le groupe de journaux a été créé avec succès. L'utilisation de `logger.info` permet de garder une trace des événements importants, ce qui est crucial pour le débogage et la maintenance des systèmes en production.

### Contexte et Importance

Gérer les journaux de manière efficace est essentiel pour toute application déployée dans le cloud. Les logs permettent de suivre l'état de l'application, d'identifier des erreurs, et d'analyser le comportement des utilisateurs. Dans le cas d'AWS Lambda, où les fonctions peuvent être exécutées de manière sporadique ou à la demande, il est crucial de conserver les logs pour chaque invocation.

- **Surveillance et Débogage** : En consignant les logs, vous pouvez surveiller les performances de votre fonction Lambda et détecter rapidement les problèmes. Par exemple, si une fonction échoue, les logs peuvent fournir des détails sur l'erreur.

- **Conformité et Audit** : Pour de nombreuses entreprises, il est impératif de conserver des traces de l'activité des applications pour des raisons de conformité. Les groupes de journaux dans CloudWatch peuvent être utilisés pour conserver un historique complet des actions effectuées par votre fonction Lambda.

- **Optimisation des Coûts** : En gérant judicieusement les groupes de journaux (par exemple, en les créant uniquement lorsque cela est nécessaire), vous pouvez éviter des coûts inutiles associés à la conservation des données dans AWS.

### Conclusion

Ce code est un bon exemple de la manière dont vous pouvez interagir avec AWS CloudWatch Logs pour créer et gérer des groupes de journaux de manière programmatique. En utilisant Boto3, vous pouvez automatiser la gestion des logs, ce qui est un aspect essentiel pour maintenir des applications cloud robustes et évolutives. Si vous avez des questions spécifiques sur ce code ou sur la gestion des logs en général, n'hésitez pas à demander !

Next we create the function requirements for IAM role and policies using the support function `create_lambda_role` and create the lambda using the support function `create_lambda` both from the `agent.py` file

In [ ]:
lambda_iam_role = create_lambda_role(agent_name)

In [ ]:
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

### Creating Agent

Now that we have created the lambda function, let's create our Agent.

To do so, we first need to create an agent role and its required policies:
* Invoke model

Let's do so using the `create_agent_role` function from the `agent.py` file.

In [ ]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

With the Agent IAM role created, we can now use the boto3 function [`create_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html) to create our agent.

On the agent creation, all you need to provide is the agent name, foundation model and instruction. We will associate an action group to the agent once it has been created

We will retrieve the `agentId`. It will be used to associate the action group to the agent in our next step.

In [ ]:
response = bedrock_agent_client.list_agents()
agent_id_list = [
    agent_summary["agentId"] for agent_summary in response["agentSummaries"] if agent_summary["agentName"] == agent_name
]
if agent_id_list:
    logger.info(f"Agent {agent_name} already exists, updating it")
    agent_id = agent_id_list[0]
    response = bedrock_agent_client.update_agent(
        agentId=agent_id_list[0],
        agentName=agent_name,
        agentResourceRoleArn=agent_role["Role"]["Arn"],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
else:
    logger.info(f"Creating agent {agent_name}")
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
    logger.info(f"Agent {agent_name} created")
    agent_id = response["agent"]["agentId"]
time.sleep(10)

Le code que vous avez partagé concerne la gestion des agents dans un service, probablement lié à AWS Bedrock, qui est utilisé pour créer et gérer des agents basés sur des modèles d'IA. Voici une analyse détaillée et des explications sur le fonctionnement de ce code.

### Explication du Code

1. **Lister les Agents Existants** :
   ```python
   response = bedrock_agent_client.list_agents()
   ```
   - Cette ligne appelle la méthode `list_agents()` pour récupérer une liste d'agents existants. La réponse contient des résumés des agents, qui incluent des détails comme l'ID et le nom de chaque agent.

2. **Filtrer les Agents par Nom** :
   ```python
   agent_id_list = [
       agent_summary["agentId"] for agent_summary in response["agentSummaries"] if agent_summary["agentName"] == agent_name
   ]
   ```
   - Ici, une liste `agent_id_list` est créée, qui contient les ID des agents dont le nom correspond à `agent_name`. Cette compréhension de liste permet de filtrer rapidement les agents existants.

3. **Mise à Jour ou Création d'un Agent** :
   ```python
   if agent_id_list:
       logger.info(f"Agent {agent_name} already exists, updating it")
       agent_id = agent_id_list[0]
       response = bedrock_agent_client.update_agent(
           agentId=agent_id_list[0],
           agentName=agent_name,
           agentResourceRoleArn=agent_role["Role"]["Arn"],
           description=agent_description,
           idleSessionTTLInSeconds=1800,
           foundationModel=agent_foundation_model,
           instruction=agent_instruction,
       )
   else:
       logger.info(f"Creating agent {agent_name}")
       response = bedrock_agent_client.create_agent(
           agentName=agent_name,
           agentResourceRoleArn=agent_role['Role']['Arn'],
           description=agent_description,
           idleSessionTTLInSeconds=1800,
           foundationModel=agent_foundation_model,
           instruction=agent_instruction,
       )
       logger.info(f"Agent {agent_name} created")
       agent_id = response["agent"]["agentId"]
   ```
   - Si `agent_id_list` n'est pas vide, cela signifie qu'un agent avec ce nom existe déjà. Le code met alors à jour cet agent en appelant `update_agent()` avec les nouveaux paramètres.
   - Si l'agent n'existe pas, il le crée en appelant `create_agent()`, puis enregistre l'ID de l'agent nouvellement créé.

4. **Pause** :
   ```python
   time.sleep(10)
   ```
   - Cette ligne fait une pause de 10 secondes après l'opération. Cela peut être utile pour s'assurer que l'agent est complètement créé ou mis à jour avant de procéder à d'autres opérations.

### Contexte et Importance

Ce code est essentiel dans un environnement où des agents intelligents doivent être gérés efficacement. Voici quelques points clés :

- **Mise à Jour Dynamique** : La possibilité de mettre à jour un agent existant est cruciale pour gérer des modèles d'IA, qui peuvent nécessiter des ajustements constants basés sur les performances ou les données d'entrée.

- **Logique de Création** : La vérification de l'existence d'un agent avant de le créer évite les erreurs et les duplications, ce qui peut entraîner des coûts supplémentaires ou des conflits dans le service.

- **Gestion des Rôles** : La gestion des rôles (`agentResourceRoleArn`) permet de s'assurer que l'agent a les autorisations nécessaires pour effectuer ses tâches, ce qui est un aspect clé de la sécurité dans les environnements cloud.

### Conclusion

Ce code démontre une approche pratique pour gérer des agents dans un service cloud, en utilisant les capacités de l'API d'AWS. En permettant des mises à jour dynamiques et en évitant les duplications, ce type de gestion est essentiel pour maintenir l'efficacité et la flexibilité des systèmes basés sur l'IA. Si vous avez d'autres questions sur ce code ou sur des sujets connexes en Python, n'hésitez pas à demander !

#### Create Agent Action Group

Now that we have created the agent, let's create an [Action Group](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html) and associate with the agent. The action group will allow our agent to get weather. To do so, we will "inform" our agent of existing functionalities using a [function schema](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html) defined in `JSON` format.

The function schema requires the function `name`, `description` and `parameters` to be provided. Each parameter has a parameter name, description, type and a boolean flag indicating if the parameter is required.

Let's define the functions `JSON` as `agent_functions`

In [ ]:
agent_functions = [
    {
        'name': 'get_weather',
        'description': 'Give the weather for a city',
        'parameters': {
            "city": {
                "description": "The city to get the weather for",
                "required": True,
                "type": "string"
            }
        }
    },
]

Now we can use the [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function from the boto3 SDK to create the action group

In [ ]:
# Check if the agent action group already exists
agent_version = "DRAFT"
response = bedrock_agent_client.list_agent_action_groups(agentId=agent_id, agentVersion=agent_version)
action_group_id_list = [
    action_group["actionGroupId"]
    for action_group in response["actionGroupSummaries"]
    if action_group["actionGroupName"] == agent_action_group_name
]

# Update or create the agent action group
if action_group_id_list:
    logger.info(f"Agent action group {agent_action_group_name} already exists, updating it")
    agent_action_group_id = action_group_id_list[0]
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        agentVersion=agent_version,
        actionGroupExecutor={"lambda": lambda_function["FunctionArn"]},
        actionGroupName=agent_action_group_name,
        functionSchema={"functions": agent_functions},
        description=agent_action_group_description,
        actionGroupId=agent_action_group_id
    )
else:
    logger.info(f"Creating agent action group {agent_action_group_name}")
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
        agentId=agent_id,
        agentVersion=agent_version,
        actionGroupExecutor={"lambda": lambda_function["FunctionArn"]},
        actionGroupName=agent_action_group_name,
        functionSchema={"functions": agent_functions},
        description=agent_action_group_description,
    )

Le code que vous avez partagé concerne la gestion des groupes d'actions pour un agent dans un environnement utilisant AWS Bedrock. Voici une analyse détaillée et une explication des différentes sections du code.

### Explication du Code

1. **Vérification de l'Existence du Groupe d'Actions** :
   ```python
   agent_version = "DRAFT"
   response = bedrock_agent_client.list_agent_action_groups(agentId=agent_id, agentVersion=agent_version)
   action_group_id_list = [
       action_group["actionGroupId"]
       for action_group in response["actionGroupSummaries"]
       if action_group["actionGroupName"] == agent_action_group_name
   ]
   ```
   - Cette partie du code vérifie si un groupe d'actions pour un agent donné existe déjà. La méthode `list_agent_action_groups` récupère tous les groupes d'actions associés à un agent spécifique (identifié par `agent_id`) et sa version (`agent_version`).
   - Ensuite, une liste `action_group_id_list` est créée, contenant les identifiants des groupes d'actions dont le nom correspond à `agent_action_group_name`.

2. **Mise à Jour ou Création du Groupe d'Actions** :
   ```python
   if action_group_id_list:
       logger.info(f"Agent action group {agent_action_group_name} already exists, updating it")
       agent_action_group_id = action_group_id_list[0]
       agent_action_group_response = bedrock_agent_client.update_agent_action_group(
           agentId=agent_id,
           agentVersion=agent_version,
           actionGroupExecutor={"lambda": lambda_function["FunctionArn"]},
           actionGroupName=agent_action_group_name,
           functionSchema={"functions": agent_functions},
           description=agent_action_group_description,
           actionGroupId=agent_action_group_id
       )
   else:
       logger.info(f"Creating agent action group {agent_action_group_name}")
       agent_action_group_response = bedrock_agent_client.create_agent_action_group(
           agentId=agent_id,
           agentVersion=agent_version,
           actionGroupExecutor={"lambda": lambda_function["FunctionArn"]},
           actionGroupName=agent_action_group_name,
           functionSchema={"functions": agent_functions},
           description=agent_action_group_description,
       )
   ```
   - Si la liste `action_group_id_list` n'est pas vide, cela signifie qu'un groupe d'actions avec ce nom existe déjà. Le code met à jour ce groupe en utilisant la méthode `update_agent_action_group`, en fournissant les détails nécessaires tels que le nom, la description, et le schéma des fonctions.
   - Si le groupe d'actions n'existe pas, le code crée un nouveau groupe d'actions avec `create_agent_action_group`. Les détails fournis sont similaires à ceux de la mise à jour, mais sans l'identifiant du groupe.

### Points Clés

- **Gestion des Groupes d'Actions** : Ce code illustre comment gérer les groupes d'actions des agents de manière efficace, en vérifiant d'abord leur existence avant d'effectuer des mises à jour ou des créations. Cela évite les erreurs potentielles et assure que les groupes d'actions sont toujours à jour.

- **Utilisation des Services AWS** : Le code utilise des fonctionnalités des services AWS, en particulier les agents et les fonctions Lambda. Cela montre comment les applications peuvent interagir avec les ressources cloud pour créer des solutions dynamiques et évolutives.

- **Log des Activités** : L'utilisation de `logger.info` pour enregistrer des messages d'information aide à garder une trace des opérations effectuées, ce qui est important pour le débogage et le suivi des événements dans des systèmes complexes.

### Conclusion

Ce code est un bon exemple de la façon de gérer les groupes d'actions des agents dans un environnement basé sur le cloud, en s'assurant que les opérations sont exécutées de manière sécurisée et efficace. Si vous avez d'autres questions sur ce code ou d'autres aspects de Python ou d'AWS, n'hésitez pas à les poser !

#### Allowing bedrock to invoke lambda function

The last requirement is to add the [resource-based policy](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda) to allow bedrock to invoke the action group lambda function.

In [ ]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda',)
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except lambda_client.exceptions.ResourceConflictException as e:
    print(e)

Le code que vous avez partagé concerne l'ajout de permissions à une fonction AWS Lambda pour permettre à un service spécifique, ici Bedrock, d'invoquer cette fonction. Voici une analyse détaillée du code :

### Explication du Code

1. **Création d'un Client Lambda** :
   ```python
   lambda_client = boto3.client('lambda')
   ```
   - Cette ligne crée un client pour interagir avec le service AWS Lambda en utilisant la bibliothèque `boto3`, qui est l'interface Python pour AWS.

2. **Ajout d'une Permission** :
   ```python
   try:
       response = lambda_client.add_permission(
           FunctionName=lambda_function_name,
           StatementId=f'allow_bedrock_{agent_id}',
           Action='lambda:InvokeFunction',
           Principal='bedrock.amazonaws.com',
           SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
       )
       print(response)
   ```
   - Le bloc `try` est utilisé pour tenter d'ajouter une permission à la fonction Lambda. Voici les paramètres utilisés :
     - **FunctionName** : le nom de la fonction Lambda à laquelle la permission est ajoutée (défini par `lambda_function_name`).
     - **StatementId** : un identifiant unique pour cette déclaration de permission. Il doit être unique dans le contexte de la fonction.
     - **Action** : l'action autorisée, ici `lambda:InvokeFunction`, qui permet d'invoquer la fonction.
     - **Principal** : le service ou l'entité à laquelle vous accordez la permission. Dans ce cas, c'est `bedrock.amazonaws.com`.
     - **SourceArn** : l'Amazon Resource Name (ARN) qui identifie la ressource autorisée à invoquer la fonction Lambda. Cela inclut des détails tels que la région et l'identifiant du compte AWS.

3. **Gestion des Exceptions** :
   ```python
   except lambda_client.exceptions.ResourceConflictException as e:
       print(e)
   ```
   - Si une exception de type `ResourceConflictException` est levée (par exemple, si la permission existe déjà), le message d'erreur est imprimé.

### Points Clés

- **Gestion des Permissions** : Le code montre comment gérer les permissions d'invocation pour les fonctions Lambda, ce qui est essentiel pour contrôler l'accès et garantir la sécurité.

- **Utilisation de `boto3`** : La bibliothèque `boto3` facilite les interactions avec les services AWS, rendant la gestion des ressources AWS plus intuitive pour les développeurs Python.

- **Simplicité et Clarté** : L'utilisation de `try` et `except` permet de gérer les erreurs de manière élégante, ce qui est crucial dans les applications qui interagissent avec des services externes comme AWS.

### Conclusion

Ce code est un bon exemple de la façon de configurer les permissions d'invocation d'une fonction AWS Lambda pour un service spécifique. Cela permet une intégration fluide avec d'autres services AWS tout en respectant les principes de sécurité.

#### Preparing agent

Before invoking the agent we need to prepare it. Preparing your agent will package all its components, including the security configurations. It will bring the agent into a state where it can be tested in runtime. We will use the [`prepare_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/prepare_agent.html) function from the boto3 sdk to prepare our agent.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(20)

### Invoking Agent

Now that our Agent is ready to be used, let's test it. To do so we will use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

To invoke an agent, you have to refer to its alias. You can create a new alias, or you can use the test alias to invoke your `DRAFT` agent. The test alias used to invoke the draft agent is `TSTALIASID` and it will work with any agent.


We will use the support function called `invoke_agent_helper` from the `agents.py` support file to allow us to invoke the agent with or without trace enabled and with or without session state. We will getinto more details about those concepts in the `03_invoke_agent.ipynb` notebook.

In [ ]:
alias_id = "TSTALIASID"

ℹ️ We can use session state (not changing `session_id`) to store information about the conversation and use it in the next invocations. This is useful when you want to keep track of the conversation context.

In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
# session_id: str = str(uuid.uuid1())
query = "Quelle est la météo à Paris ?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False)
print(response)

In [ ]:
# session_id: str = str(uuid.uuid1())
query = "Quelle est la météo à ?"
response = invoke_agent_helper(
    query, session_id, agent_id, alias_id, enable_trace=True
)

In [ ]:
# session_id: str = str(uuid.uuid1())
query = "Qui est Barack Obama ?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False)
print(response)